In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [101]:
player_data = pd.read_csv('..\\Consolidated Data\\Player_Attribute_Data.csv')
team_rankings = pd.read_csv('..\\Consolidated Data\\Team Rankings_corrected.csv')

In [102]:
team_rankings=team_rankings.drop(['Unnamed: 0'],axis=1)

In [103]:
player_df = player_data[['sofifa_id','club', 'overall','position', 'section', 'role']]

In [104]:
combo_df = pd.merge(player_df,team_rankings,how='inner',left_on='club',right_on='Team')

In [105]:
combo_df=combo_df.drop(['sofifa_id','club','Team'],axis=1)

In [106]:
role_df = combo_df.groupby(['role','Tier']).mean()

In [107]:
role_df.reset_index(inplace = True)

In [210]:
def club_role(club):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    club_df = player_df[player_df['club']==club][['overall','section','role']]
    club_df = club_df.groupby(['role']).mean()
    club_df = club_df.reset_index()
    upper_tier_df = role_df[role_df['Tier']==(club_tier-1)]
    upper_tier_df = upper_tier_df.reset_index()
    upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
    role_s_df = pd.merge(upper_tier_df,club_df,how='inner',on='role')
    role_s_df.columns = ['Role','Upper_Tier_Rating','Club_Rating']
    role_s_df['Rating_Gap'] = role_s_df['Upper_Tier_Rating'] - role_s_df['Club_Rating']
#     role_focus = role_s_df[role_s_df['Rating_Gap']==role_s_df['Rating_Gap'].values.max()]['Role'].values[0]
    role_focus = role_s_df[role_s_df['Rating_Gap']==role_s_df['Rating_Gap']]
    return role_focus

In [215]:
x= club_role('Manchester City')

In [216]:
print(x)

      Role  Upper_Tier_Rating  Club_Rating  Rating_Gap
0   Attack          80.818182    79.500000    1.318182
1   Defend          78.360656    75.210526    3.150129
2  Support          78.428571    79.000000   -0.571429


In [228]:
def club_section(club,role):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]    
    club_df = player_df[player_df['club']==club][['overall','section','role']]
    c_sec_df = club_df[club_df['role']==role]
    c_sec_df = c_sec_df.groupby(['section']).mean()
    c_sec_df = c_sec_df.reset_index()
    upper_tier_df = combo_df[combo_df['Tier']==club_tier-1]
    upper_role_df = upper_tier_df[upper_tier_df['role']==role]
    upper_role_df = upper_role_df[['section','overall']]
    upper_role_df = upper_role_df.groupby(['section']).mean()
    upper_role_df = upper_role_df.reset_index()
    section_df = pd.merge(upper_role_df,c_sec_df,how='inner',on='section')
    section_df.columns = ['Section','Upper_Tier_Rating','Club_Rating']
    section_df['Rating_Gap'] = section_df['Upper_Tier_Rating']-section_df['Club_Rating']
#     section_focus = section_df['Section'].values[0]
    return section_df

In [229]:
y = club_section('Manchester United','Support')

In [230]:
y

,Section,Upper_Tier_Rating,Club_Rating,Rating_Gap
0,Midfield,78.428571,79,-0.571429


In [223]:
def club_position(club,role,section):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    club_df = player_df[player_df['club']==club][['overall','position','section','role']]
    c_role_df = club_df[club_df['role']==role]
    c_sec_df = c_role_df[c_role_df['section']==section]
    c_pos_df = c_sec_df.groupby(['position']).mean()
    c_pos_df = c_pos_df.reset_index()
    upper_tier_df = combo_df[combo_df['Tier']==club_tier-1]
    upper_role_df = upper_tier_df[upper_tier_df['role']==role]
    upper_sec_df = upper_role_df[upper_role_df['section']==section]
    upper_pos_df = upper_role_df[['position','overall']]
    upper_pos_df = upper_pos_df.groupby(['position']).mean()
    upper_pos_df = upper_pos_df.reset_index()
    position_df = pd.merge(upper_pos_df,c_pos_df,how='inner',on='position')
    position_df.columns=['Position','Upper_Tier_Rating','Club_Rating']
    position_df['Rating_Gap'] = position_df['Upper_Tier_Rating']-position_df['Club_Rating']
    return position_df

In [224]:
z = club_position('Arsenal','Defend','Defence')

In [225]:
z

,Position,Upper_Tier_Rating,Club_Rating,Rating_Gap
0,CB,78.588235,75.833333,2.754902
1,LB,72.882353,78.000000,-5.117647
2,RB,76.882353,80.000000,-3.117647


In [244]:
x= club_role('Manchester City')

In [270]:
x=x.sort_values(by='Rating_Gap', ascending=False)

In [271]:
x=x.reset_index()

In [272]:
x=x.drop(['index'],axis=1)

In [273]:
x.iloc[0]['Role']

'Defend'

In [274]:
x

,Role,Upper_Tier_Rating,Club_Rating,Rating_Gap
0,Defend,78.360656,75.210526,3.150129
1,Attack,80.818182,79.500000,1.318182
2,Support,78.428571,79.000000,-0.571429


In [275]:
roles = x['Role']

In [276]:
roles

0     Defend
1     Attack
2    Support
Name: Role, dtype: object

In [303]:
for i in range(0,len(roles)):
    print()
    print('Role:',x.iloc[i]['Role'],' --> ',x.iloc[i]['Rating_Gap'])
    print()
    section_df=club_section('Manchester City',roles[i])
    section_df=section_df.sort_values(by='Rating_Gap',ascending=False)
    for j in range(0,len(section_df)):
        print('Section: ',section_df.iloc[j]['Section'],' --> ',section_df.iloc[j]['Rating_Gap'])
        position_df=club_position('Manchester City',roles[i],section_df.iloc[j]['Section'])
        for k in range(0,len(position_df)):
            print(position_df['Position'].values[k],' :',position_df['Rating_Gap'].values[k])
#             print(position_df['Rating_Gap'].values[0])
        print()


Role: Defend  -->  3.1501294219154374

Section:  Goalkeeper  -->  5.799999999999997
GK  : 5.799999999999997

Section:  Defence  -->  2.4363636363636374
CB  : 2.7238095238095212
LB  : 6.194444444444443
RB  : -5.200000000000003

Section:  Midfield  -->  0.6666666666666714
CDM  : 0.6666666666666714


Role: Attack  -->  1.318181818181813

Section:  Forward  -->  1.4903846153846132
LW  : 8.0
RW  : -4.583333333333329
ST  : 1.7777777777777857

Section:  Midfield  -->  0.4642857142857082
CAM  : 0.4642857142857082


Role: Support  -->  -0.5714285714285694

Section:  Midfield  -->  -0.5714285714285694
CM  : 0.5



In [265]:
for i in range(0,len(x)):
    roles = x.iloc[i]['Role']
    print(role)
    y = club_section('Manchester City',role)
    section=(y['Section'])
    print(section)

Defend
0       Defence
1    Goalkeeper
2      Midfield
Name: Section, dtype: object
Attack
0     Forward
1    Midfield
Name: Section, dtype: object
Support
0    Midfield
Name: Section, dtype: object
